In [3]:
!wget https://huggingface.co/datasets/Libosa2707/vietnamese-poem/resolve/main/poems_dataset.csv
!wget https://huggingface.co/datasets/phamson02/vietnamese-poetry-corpus/resolve/main/poems_dataset.csv

--2024-08-07 09:07:55--  https://huggingface.co/datasets/Libosa2707/vietnamese-poem/resolve/main/poems_dataset.csv
Resolving huggingface.co (huggingface.co)... 18.239.69.83, 18.239.69.50, 18.239.69.31, ...
Connecting to huggingface.co (huggingface.co)|18.239.69.83|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f1/ab/f1ab000118e9cde67ceda9e6bc227f8bf8dfa736de9307d30ea2f95af2ebf69c/0248dcd6e6442c76f66b61088dc6f4cee92b62f386bdbae649a248f9aa17df80?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27poems_dataset.csv%3B+filename%3D%22poems_dataset.csv%22%3B&response-content-type=text%2Fcsv&Expires=1723280875&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMzI4MDg3NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mMS9hYi9mMWFiMDAwMTE4ZTljZGU2N2NlZGE5ZTZiYzIyN2Y4YmY4ZGZhNzM2ZGU5MzA3ZDMwZWEyZjk1YWYyZWJmNjljLzAyNDhkY2Q2ZTY0NDJjNzZmNjZiNjEwODhkYzZmNGNlZTk

In [3]:
!pip install underthesea


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 77.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.2 MB/s eta 0:00:00


In [4]:
import os
import re
import time
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from underthesea import word_tokenize
import pickle

In [6]:
def save_pkl(save_object, save_file):
    with open(save_file, 'wb') as f:
        pickle.dump(save_object, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_pkl(load_file):
    with open(load_file, 'rb') as f:
        output = pickle.load(f)
    return output

In [7]:
data = pd.read_csv("poems_dataset.csv")

In [8]:
data.head()

,id,content,title,url,genre
0,139248,ngày đông se sắt lạnh trong lòng\ncó việc đi n...,SAY NẮNG,https://www.facebook.com/groups/48640773509859...,7 chu
1,139249,ôm đàn thao thức đến nữa đêm\nréo rắt cung âm ...,NaN,https://www.facebook.com/groups/17645444269765...,7 chu
2,139250,tết có người vui có kẻ buồn\nngười cười toe to...,TẾT HAI THÁI CỰC,https://www.facebook.com/groups/17645444269765...,7 chu
3,139251,đã quá ba mươi mộng lỡ làng\nđi tìm day dứt mả...,TRÁI NGANG ĐỨC HẠNH,https://www.facebook.com/groups/48640773509859...,7 chu
4,139252,mai đào nở rộ đón nàng xuân\nsợi nắng hanh vàn...,DÁNG XUÂN,https://www.facebook.com/groups/17645444269765...,7 chu


In [9]:
train_df, val_df = train_test_split(data, test_size=0.3, random_state=42)

In [10]:
train_documents = [doc for doc in train_df['content'].tolist()]
val_documents = [doc for doc in val_df['content'].tolist()]

In [7]:
class Vocabulary:
    """ The Vocabulary class is used to record words, which are used to convert
        text to numbers and vice versa.
    """
    def __init__(self):
        self.word2id = dict()

        self.pad_id = 0
        self.unk_id = 1
        self.sos_id = 2
        self.eos_id = 3

        self.word2id['<pad>'] = self.pad_id # Pad Token
        self.word2id['<unk>'] = self.unk_id # Unknown Token
        self.word2id['<s>'] = self.sos_id # start token
        self.word2id['</s>'] = self.eos_id # end token

        self.id2word = {v: k for k, v in self.word2id.items()}

    def __getitem__(self, word):
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        return word in self.word2id

    def __len__(self):
        return len(self.word2id)

    def lookup_tokens(self, word_indexes: list):
        """
        @param word_indexes (list(int))
        @return words (list(str))
        """
        return [self.id2word[word_index] for word_index in word_indexes]

    def add(self, word):
        """ Add word to vocabulary
        @param word (str)
        @return index (str): index of the word just added
        """
        if word not in self:
            word_index = self.word2id[word] = len(self.word2id)
            self.id2word[word_index] = word
            return word_index
        else:
            return self[word]

    def corpus_to_tensor(self, corpus, is_tokenized=False):
        """ Convert corpus to a list of indices tensor
        @param corpus (list(str) if is_tokenized==False else list(list(str)))
        @param is_tokenized (bool)
        @return indicies_corpus (list(tensor))
        """
        if is_tokenized:
            tokenized_corpus = corpus
        else:
            tokenized_corpus = self.tokenize_corpus(corpus)
        indicies_corpus = list()
        for document in tqdm(tokenized_corpus):
            indicies_document = torch.tensor(list(map(lambda word: self[word], document)), dtype=torch.long)
            indicies_corpus.append(indicies_document)

        return indicies_corpus

    def tensor_to_corpus(self, tensor):
        """ Convert list of indices tensor to a list of tokenized documents
        @param indicies_corpus (list(tensor))
        @return corpus (list(list(str)))
        """
        corpus = list()
        for indicies in tqdm(tensor):
            document = list(map(lambda index: self.id2word[index.item()], indicies))
            corpus.append(document)

        return corpus

    @staticmethod
    def tokenize_corpus(corpus):
        """Split the documents of the corpus into words
        @param corpus (list(str)): list of documents
        @return tokenized_corpus (list(list(str))): list of words
        """
        print("Tokenize the corpus...")
        tokenized_corpus = list()
        for document in tqdm(corpus):
            tokenized_document = ['<s>'] + re.findall(r'(\w+|[^\w\s]|\S+|\n)', document) + ['</s>']
            tokenized_corpus.append(tokenized_document)

        return tokenized_corpus

    @classmethod
    def from_documents(cls, documents):
        words = set(word for doc in documents for word in re.findall(r'\w+|\S|\n', doc))
        vocab = cls()
        for w in words:
            vocab.add(w)
        return vocab

    @classmethod
    def from_pretrained(cls, save_dir):
        import pickle
        """
        Initialize a new Vocabulary from a pre-trained vocabulary saved as a pickle file.

        Parameters:
        - vocab_file (str): Path to the pickle file containing the pre-trained vocabulary.

        Returns:
        - vocab (Vocabulary): Initialized Vocabulary object.
        """
        with open(os.path.join(save_dir, "vocab.pkl"), 'rb') as file:
            pretrained_vocab = pickle.load(file)

        return cls.init_vocab_from_pretrained(pretrained_vocab)

    @staticmethod
    def init_vocab_from_pretrained(pretrained_vocab):
        """
        Initialize a new Vocabulary from a pre-trained vocabulary.

        Parameters:
        - pretrained_vocab (dict): A dictionary mapping words to their corresponding indices.

        Returns:
        - vocab (Vocabulary): Initialized Vocabulary object.
        """
        vocab = Vocabulary()

        # Copy the pre-trained word-to-index mapping to the new vocabulary
        vocab.word2id.update(pretrained_vocab)

        # Create the index-to-word mapping
        vocab.id2word = {v: k for k, v in vocab.word2id.items()}

        return vocab

    def save_pretrained(self, save_dir):
        import os
        import pickle
        """
        Save the vocabulary to a pickle file.

        Parameters:
        - pickle_file_path (str): Path to the pickle file where the vocabulary will be saved.
        """
        os.makedirs(save_dir, exist_ok=True)
        with open(os.path.join(save_dir, "vocab.pkl"), 'wb') as file:
            pickle.dump(self.word2id, file)

In [12]:
vocab = Vocabulary.from_documents(train_documents)

In [8]:
from torch.utils.data import Dataset, DataLoader

class PoemGenerationDataset(Dataset):
    def __init__(self, documents, vocab, max_length=None):
        self.vocab = vocab
        self.sos_idx = vocab["<s>"]
        self.eos_idx = vocab["</s>"]
        self.pad_idx = vocab["<pad>"]
        self.documents = documents
        self.max_length = max_length
        self.tokenized_documents = self.vocab.tokenize_corpus(self.documents)
        self.tensor_data = self.vocab.corpus_to_tensor(self.tokenized_documents, is_tokenized=True)

    def __len__(self):
        return len(self.tensor_data)

    def __getitem__(self, idx):
        return self.tensor_data[idx]

    def shift_right(self, input_ids, pad_token=0):
        """
        Shifts the input tensor of batch of input ids to the right by one position.

        Args:
        - input_ids (torch.Tensor): Batch of input ids with shape (batch_size, sequence_length).
        - pad_token (int): Token used for padding. Defaults to 0.

        Returns:
        - torch.Tensor: Shifted input tensor with the same shape as input_ids.
        """
        # Create a tensor of the same shape filled with pad_token
        padding_column = torch.full_like(input_ids[:, :1], pad_token)

        # Concatenate the padding column to the left of the input_ids
        shifted_ids = torch.cat([padding_column, input_ids[:, :-1]], dim=-1)

        return shifted_ids


    def collate_fn(self, examples):
        examples = sorted(examples, key=lambda e: len(e), reverse=True)

        if self.max_length is not None:
            examples = [torch.cat([e[:self.max_length-1], torch.tensor([self.eos_idx])]) for e in examples]

        docs = [e for e in examples]
        # padding tokens
        input_ids = torch.nn.utils.rnn.pad_sequence(docs, batch_first=True, padding_value=self.pad_idx)
        labels = input_ids.clone()
        input_ids = self.shift_right(input_ids, pad_token=self.pad_idx)
        return {"inputs": input_ids, "labels": labels}

In [14]:
train_dataset = PoemGenerationDataset(train_documents, vocab, max_length=512)
val_dataset = PoemGenerationDataset(val_documents, vocab, max_length=512)

Tokenize the corpus...


100%|██████████| 119827/119827 [00:08<00:00, 14959.43it/s]


Tokenize the corpus...


100%|██████████| 51355/51355 [00:03<00:00, 16340.31it/s]


In [15]:
train_dataloader = DataLoader(train_dataset, batch_size=5, collate_fn=train_dataset.collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=5, collate_fn=val_dataset.collate_fn)

In [12]:
class LSTMForPoemGeneration(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embedding = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.padding_idx)
        self.batch_norm = nn.BatchNorm1d(config.hidden_size)
        self.lstm = nn.LSTM(config.hidden_size,
                            config.hidden_size,
                            num_layers=config.num_layers,
                            dropout=config.dropout,
                            batch_first=True)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, input_ids, labels=None):
        embeds = self.dropout(self.embedding(input_ids))
        bn_output = self.batch_norm(embeds.permute(0,2,1)).permute(0,2,1)
        output, (hidden, cell) = self.lstm(bn_output)

        logits = self.lm_head(self.dropout(output))
        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            labels = labels.to(logits.device)
            loss = loss_fct(logits.view(-1, logits.size(2)), labels.view(-1))
        return logits, loss

    @torch.no_grad()
    def generate(self, input_ids, max_length=100, temperature=1.0):
        self.to(input_ids.device)
        self.eval()

        current_length = input_ids.size(1)
        while current_length < max_length:
            logits, _ = self.forward(input_ids)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            current_length += 1

            # Check if </s> token is generated
            if next_token_id.item() == vocab.eos_id:
                break

        return input_ids

    def save_pretrained(self, save_dir):
        os.makedirs(save_dir, exist_ok=True)
        torch.save(self.config, os.path.join(save_dir, "config.pt"))
        torch.save(self.state_dict(), os.path.join(save_dir, "pytorch_model.pt"))

    @classmethod
    def from_pretrained(cls, saved_dir):
        config = torch.load(os.path.join(saved_dir, "config.pt"))
        model = cls(config)
        model.load_state_dict(torch.load(os.path.join(saved_dir, "pytorch_model.pt")))
        return model

In [11]:
from dataclasses import dataclass

@dataclass
class LSTMConfig:
    vocab_size = len(vocab)
    hidden_size = 512
    padding_idx = 0
    dropout = 0.05
    num_layers = 2

config = LSTMConfig()

In [18]:
# def prepare_model(config, vocab, *, w2v=None):
#     # Generation Model
#     model = LSTMForPoemGeneration(config)

#     if w2v is not None:
#         for w, i in w2v.stoi.items():
#             model.embedding.weight.data[vocab[w]].copy_(w2v.vectors[i])

#         model.embedding.weight.data[vocab.unk_id] = torch.zeros(config.hidden_size)
#         model.embedding.weight.data[vocab.pad_id] = torch.zeros(config.hidden_size)

#     return model

In [19]:
# model = prepare_model(config, vocab)
model = LSTMForPoemGeneration(config)

In [20]:
def train(model, train_dataloader, val_dataloader, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

    best_val_loss = float('inf')
    consecutive_no_improvement = 0

    for epoch in range(args.num_epochs):
        total_loss = 0.0
        model.train()

        for batch in (pbar:=tqdm(train_dataloader, desc=f"(train) Epoch {epoch}")):
            inputs, labels = batch['inputs'].to(device), batch['labels'].to(device)

            optimizer.zero_grad()

            logits, loss = model(inputs, labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
            optimizer.step()

            pbar.set_postfix(loss=loss.item())
            total_loss += loss.item()

        average_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch}/{args.num_epochs}, Train Loss: {average_loss:.4f}")

        # Validation
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for val_batch in (pbar:=tqdm(val_dataloader, desc=f"(val) Epoch {epoch}")):
                val_inputs, val_labels = val_batch['inputs'].to(device), val_batch['labels'].to(device)
                val_logits, val_loss_batch = model(val_inputs, val_labels)
                pbar.set_postfix(loss=val_loss_batch.item())
                val_loss += val_loss_batch.item()

        average_val_loss = val_loss / len(val_dataloader)
        print(f"Epoch {epoch}/{args.num_epochs}, Validation Loss: {average_val_loss:.4f}")

        # Early stopping check
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            consecutive_no_improvement = 0
            print(f"Save epoch {epoch} checkpoint")
            # torch.save({"state_dict": model.state_dict(), "vocab": vocab}, f"checkpoint_{epoch}.pt")
            model.save_pretrained(f"checkpoint_{epoch}")
            vocab.save_pretrained(f"checkpoint_{epoch}")
        else:
            consecutive_no_improvement += 1

        if consecutive_no_improvement >= args.patience:
            print(f"Early stopping after {epoch} epochs without improvement on validation set.")
            break

    print("Training complete.")

In [21]:
class TrainingArguments:
    def __init__(self):
        self.num_epochs = 5
        self.patience = 5
        self.lr = 1e-3
        self.clip = 5.0

training_args = TrainingArguments()

In [23]:
train(model, train_dataloader, val_dataloader, args=training_args)

(train) Epoch 0:   1%|          | 253/23966 [01:28<2:18:58,  2.84it/s, loss=3.16]


KeyboardInterrupt: 

# Inference

In [ ]:
class Vocabulary:
    """ The Vocabulary class is used to record words, which are used to convert
        text to numbers and vice versa.
    """
    def __init__(self):
        self.word2id = dict()

        self.pad_id = 0
        self.unk_id = 1
        self.sos_id = 2
        self.eos_id = 3

        self.word2id['<pad>'] = self.pad_id # Pad Token
        self.word2id['<unk>'] = self.unk_id # Unknown Token
        self.word2id['<s>'] = self.sos_id # start token
        self.word2id['</s>'] = self.eos_id # end token

        self.id2word = {v: k for k, v in self.word2id.items()}

    def __getitem__(self, word):
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        return word in self.word2id

    def __len__(self):
        return len(self.word2id)

    def lookup_tokens(self, word_indexes: list):
        """
        @param word_indexes (list(int))
        @return words (list(str))
        """
        return [self.id2word[word_index] for word_index in word_indexes]

    def add(self, word):
        """ Add word to vocabulary
        @param word (str)
        @return index (str): index of the word just added
        """
        if word not in self:
            word_index = self.word2id[word] = len(self.word2id)
            self.id2word[word_index] = word
            return word_index
        else:
            return self[word]

    def corpus_to_tensor(self, corpus, is_tokenized=False):
        """ Convert corpus to a list of indices tensor
        @param corpus (list(str) if is_tokenized==False else list(list(str)))
        @param is_tokenized (bool)
        @return indicies_corpus (list(tensor))
        """
        if is_tokenized:
            tokenized_corpus = corpus
        else:
            tokenized_corpus = self.tokenize_corpus(corpus)
        indicies_corpus = list()
        for document in tqdm(tokenized_corpus):
            indicies_document = torch.tensor(list(map(lambda word: self[word], document)), dtype=torch.long)
            indicies_corpus.append(indicies_document)

        return indicies_corpus

    def tensor_to_corpus(self, tensor):
        """ Convert list of indices tensor to a list of tokenized documents
        @param indicies_corpus (list(tensor))
        @return corpus (list(list(str)))
        """
        corpus = list()
        for indicies in tqdm(tensor):
            document = list(map(lambda index: self.id2word[index.item()], indicies))
            corpus.append(document)

        return corpus

    @staticmethod
    def tokenize_corpus(corpus):
        """Split the documents of the corpus into words
        @param corpus (list(str)): list of documents
        @return tokenized_corpus (list(list(str))): list of words
        """
        print("Tokenize the corpus...")
        tokenized_corpus = list()
        for document in tqdm(corpus):
            tokenized_document = ['<s>'] + re.findall(r'(\w+|[^\w\s]|\S+|\n)', document) + ['</s>']
            tokenized_corpus.append(tokenized_document)

        return tokenized_corpus

    @classmethod
    def from_documents(cls, documents):
        words = set(word for doc in documents for word in re.findall(r'\w+|\S|\n', doc))
        vocab = cls()
        for w in words:
            vocab.add(w)
        return vocab

    @classmethod
    def from_pretrained(cls, save_dir):
        import pickle
        """
        Initialize a new Vocabulary from a pre-trained vocabulary saved as a pickle file.

        Parameters:
        - vocab_file (str): Path to the pickle file containing the pre-trained vocabulary.

        Returns:
        - vocab (Vocabulary): Initialized Vocabulary object.
        """
        with open(os.path.join(save_dir, "vocab.pkl"), 'rb') as file:
            pretrained_vocab = pickle.load(file)

        return cls.init_vocab_from_pretrained(pretrained_vocab)

    @staticmethod
    def init_vocab_from_pretrained(pretrained_vocab):
        """
        Initialize a new Vocabulary from a pre-trained vocabulary.

        Parameters:
        - pretrained_vocab (dict): A dictionary mapping words to their corresponding indices.

        Returns:
        - vocab (Vocabulary): Initialized Vocabulary object.
        """
        vocab = Vocabulary()

        # Copy the pre-trained word-to-index mapping to the new vocabulary
        vocab.word2id.update(pretrained_vocab)

        # Create the index-to-word mapping
        vocab.id2word = {v: k for k, v in vocab.word2id.items()}

        return vocab

    def save_pretrained(self, save_dir):
        import os
        import pickle
        """
        Save the vocabulary to a pickle file.

        Parameters:
        - pickle_file_path (str): Path to the pickle file where the vocabulary will be saved.
        """
        os.makedirs(save_dir, exist_ok=True)
        with open(os.path.join(save_dir, "vocab.pkl"), 'wb') as file:
            pickle.dump(self.word2id, file)


def tokenize(vocab, input_text):
    tokens = ['<s>'] + re.findall(r'\w+|\S|\n', input_text)
    input_ids = torch.tensor([vocab[word] for word in tokens], dtype=torch.long).unsqueeze(0)
    return input_ids

def generate_text(model, vocab, input_text, max_length=100, temperature=1.0):
    input_ids = tokenize(vocab, input_text).to("cuda")
    output_ids = model.generate(input_ids, max_length=max_length, temperature=temperature)
    generated_tokens = [vocab.id2word[index.item()] for index in output_ids[0]]
    generated_text = ' '.join(generated_tokens[1:-1])  # Exclude <s> and </s>
    return generated_text

def generate_text_with_animation(model, vocab, input_text, max_length=100, temperature=1.0):
    import time
    input_ids = tokenize(vocab, input_text).to("cuda")
    model.to(input_ids.device)
    model.eval()

    generated_text = f"{input_text}"
    current_length = input_ids.size(1)

    print(generated_text, end=' ', flush=True)

    with torch.no_grad():
        while current_length < max_length:
            logits, _ = model(input_ids)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            current_length += 1

            # Check if </s> token is generated
            if next_token_id.item() == vocab.eos_id:
                break

            generated_token = vocab.id2word[next_token_id.item()]
            generated_text += f"{generated_token} "
            print(generated_token, end=' ', flush=True)

            # Simulate some processing time
            time.sleep(0.1)


class LSTMForPoemGeneration(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embedding = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.padding_idx)
        self.batch_norm = nn.BatchNorm1d(config.hidden_size)
        self.lstm = nn.LSTM(config.hidden_size,
                            config.hidden_size,
                            num_layers=config.num_layers,
                            dropout=config.dropout,
                            batch_first=True)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, input_ids, labels=None):
        embeds = self.dropout(self.embedding(input_ids))
        bn_output = self.batch_norm(embeds.permute(0,2,1)).permute(0,2,1)
        output, (hidden, cell) = self.lstm(bn_output)

        logits = self.lm_head(self.dropout(output))
        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            labels = labels.to(logits.device)
            loss = loss_fct(logits.view(-1, logits.size(2)), labels.view(-1))
        return logits, loss

    @torch.no_grad()
    def generate(self, input_ids, max_length=100, temperature=1.0):
        self.to(input_ids.device)
        self.eval()

        current_length = input_ids.size(1)
        while current_length < max_length:
            logits, _ = self.forward(input_ids)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            current_length += 1

            # Check if </s> token is generated
            if next_token_id.item() == vocab.eos_id:
                break

        return input_ids

    def save_pretrained(self, save_dir):
        os.makedirs(save_dir, exist_ok=True)
        torch.save(self.config, os.path.join(save_dir, "config.pt"))
        torch.save(self.state_dict(), os.path.join(save_dir, "pytorch_model.pt"))

    @classmethod
    def from_pretrained(cls, saved_dir):
        config = torch.load(os.path.join(saved_dir, "config.pt"))
        model = cls(config)
        model.load_state_dict(torch.load(os.path.join(saved_dir, "pytorch_model.pt")))
        return model

In [5]:
def tokenize(vocab, input_text):
    tokens = ['<s>'] + re.findall(r'\w+|\S|\n', input_text)
    input_ids = torch.tensor([vocab[word] for word in tokens], dtype=torch.long).unsqueeze(0)
    return input_ids

def generate_text(model, vocab, input_text, max_length=100, temperature=1.0):
    input_ids = tokenize(vocab, input_text).to("cuda")
    output_ids = model.generate(input_ids, max_length=max_length, temperature=temperature)
    generated_tokens = [vocab.id2word[index.item()] for index in output_ids[0]]
    generated_text = ' '.join(generated_tokens[1:-1])  # Exclude <s> and </s>
    return generated_text

def generate_text_with_animation(model, vocab, input_text, max_length=100, temperature=1.0):
    import time
    input_ids = tokenize(vocab, input_text).to("cuda")
    model.to(input_ids.device)
    model.eval()

    generated_text = f"{input_text}"
    current_length = input_ids.size(1)

    print(generated_text, end=' ', flush=True)

    with torch.no_grad():
        while current_length < max_length:
            logits, _ = model(input_ids)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            current_length += 1

            # Check if </s> token is generated
            if next_token_id.item() == vocab.eos_id:
                break

            generated_token = vocab.id2word[next_token_id.item()]
            generated_text += f"{generated_token} "
            print(generated_token, end=' ', flush=True)

            # Simulate some processing time
            time.sleep(0.1)




In [29]:
# import torch
# import torch.nn.functional as F
# import re

# def tokenize(vocab, input_text):
#     tokens = ['<s>'] + re.findall(r'\w+|\S|\n', input_text)
#     input_ids = torch.tensor([vocab.get(word, vocab['<unk>']) for word in tokens], dtype=torch.long).unsqueeze(0)
#     return input_ids

# def generate_text(model, vocab, input_text, max_length=100, temperature=1.0):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     input_ids = tokenize(vocab, input_text).to(device)
#     output_ids = model.generate(input_ids, max_length=max_length, temperature=temperature)
#     generated_tokens = [vocab.id2word.get(index.item(), '<unk>') for index in output_ids[0]]
#     generated_text = ' '.join(generated_tokens[1:-1])  # Exclude <s> and </s>
#     return generated_text

# def generate_text_with_animation(model, vocab, input_text, max_length=100, temperature=1.0):
#     import time
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     input_ids = tokenize(vocab, input_text).to(device)
#     model.to(device)
#     model.eval()

#     generated_text = f"{input_text}"
#     current_length = input_ids.size(1)

#     print(generated_text, end=' ', flush=True)

#     with torch.no_grad():
#         while current_length < max_length:
#             logits, _ = model(input_ids)
#             logits = logits[:, -1, :] / temperature
#             probs = F.softmax(logits, dim=-1)
#             next_token_id = torch.multinomial(probs, num_samples=1)
#             input_ids = torch.cat([input_ids, next_token_id], dim=1)
#             current_length += 1

#             # Check if </s> token is generated
#             if next_token_id.item() == vocab.eos_id:
#                 break

#             generated_token = vocab.id2word.get(next_token_id.item(), '<unk>')
#             generated_text += f"{generated_token} "
#             print(generated_token, end=' ', flush=True)

#             # Simulate some processing time
#             time.sleep(0.1)


In [13]:
# 1. load pretrained vocab
vocab = Vocabulary.from_pretrained("/kaggle/input/lstm-gen-model-result/checkpoint_0")
model = LSTMForPoemGeneration.from_pretrained("/kaggle/input/lstm-gen-model-result/checkpoint_0")

In [14]:
import torch
print(torch.cuda.is_available())


True


In [15]:
generate_text_with_animation(model, vocab, "mừng xuân sang", max_length=512, temperature=.9)

mừng xuân sang lộc trùm lên 
 tươi cười mới rạng khoẻ vui quây quần 
 mùa xuân cùng với bão bùng 
 mừng xuân vật chất thăng trầm khó khăn 
 ngày xuân ba sáu thanh rằm 
 già già tất cả xuân đang chan hòa 
 xuân về khoe sắc xuân ba 
 hương xuân sắc thắm xuân ngời hương xuân 
 xuân xuân vui với xuân quân 
 giầu chẳng như thế bình minh chói màu 
 loài hoa đua nở nhuỵ vàng 
 xuân về khi dịch ngập tràn hoa tươi 
 xuân về đến tận chân trời 
 xuân về nghỉ tới một thời gan hoa 
 bướm vàng vui hót đông hoà 
 xuân sang nhường phật máy nhà tự do 
 
 nắng xuân dạo dưới chuông chùa 
 xuân về thêm mối vàng hoa khoe vàng 
 hoa tươi duyên dáng phượng vàng 
 ông trời mưa gió kéo sang rộn ràng 
 xuân về khoe sắc vàng vang 
 thóc đến trú quả lộc vàng lại đông 
 xuân về ong bướm ong ong 
 xuân về đâm lộc ngõ vào ong vui 
 xuân về rồi sẽ chia tay 
 trái tim giao nhịp đón chào thầm yêu 
 xuân yêu đến nở rất nhiều 
 xuân về đoàn tụ sớm chiều đu đưa 
 xuân qua xuân đến bao mươi 
 xuân về xuân đến thêu thùa bốn